# Imports

In [9]:
import calendar
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import tqdm
import imageio.v2 as imageio
import itertools
import matplotlib
import matplotlib.pyplot as plt
import branca.colormap as cm
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [16, 11] 

In [10]:
station_df = pd.read_csv('input_data/all_stations_by_hour_weekday.csv')

In [11]:
station_df = gpd.GeoDataFrame(
    station_df, geometry=gpd.points_from_xy(station_df.coordinatesX, station_df.coordinatesY))
station_df["copy_geometry"] = station_df.geometry


In [12]:
plz_shape_df = gpd.read_file('input_data/plz-5stellig.shp', dtype={'plz': str})
plz_region_df = pd.read_csv(
    'input_data/zuordnung_plz_ort.csv', 
    sep=',', 
    dtype={'plz': str}
)
plz_einwohner_df = pd.read_csv(
    'input_data/plz_einwohner.csv', 
    sep=',', 
    dtype={'plz': str, 'einwohner': int}
)

In [13]:
germany_df = pd.merge(
    left=plz_shape_df, 
    right=plz_region_df, 
    on='plz',
    how='inner'
)


In [14]:
station_df = station_df.set_crs('epsg:4236')

In [15]:
hamburg_df = germany_df.query('ort == "Hamburg"')

joined_df = hamburg_df.sjoin(station_df, how="left")
agg_df = joined_df.groupby(['plz', 'resultHour', 'resultWeekday']).average_res.sum().reset_index()
combined_df = hamburg_df.merge(agg_df, how='left', on="plz")

/Users/felixude/opt/anaconda3/envs/traffic/lib/python3.10/site-packages/geopandas/geodataframe.py:2090: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4236

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


# Creating Station Hour Plots

In [82]:
from tqdm import tqdm
hamburg_df = germany_df.query('ort == "Hamburg"')

station_df.to_csv('station_df.csv', index=False)

for day, hour in tqdm(list(itertools.product(range(1, 8, 1), range(0, 24, 1)))):

    unique_coords = station_df.query(f'resultHour == {hour} and resultWeekday == {day}')
    plt.ion()

    hamburg = hamburg_df.plot(
    )

    cur = unique_coords.plot(ax=hamburg,
                       column='average_res',
                       cmap='Reds', 
                             vmin = 1,
                             vmax = 17,
                             legend_kwds={"label": "Bikes"}


                      )

    cur.set_xlim(9.7, 10.37)
    cur.set_ylim(53.38, 53.75)
    cur.grid(False)
    cur.set_facecolor("lavender")
    w_day = calendar.day_name[day-1]
    if hour < 10:
        hour_string = "0" + str(hour)
    else:
        hour_string = str(hour)
    cur.set_title(f"Hamburg bike stations at {w_day} {hour_string}:00", fontsize=20)
    cur.tick_params(left = False, right = False , labelleft = False ,
            labelbottom = False, bottom = False)
    file_name = f"images/stations_{day}_{hour}.jpg"
    cur.figure.savefig(file_name, dpi=300, bbox_inches='tight')
    plt.close(cur.figure)




100%|██████████████████████████████████████████████████████████████████| 168/168 [00:52<00:00,  3.21it/s]


In [83]:
from tqdm import tqdm
for day, hour in tqdm(list(itertools.product(range(1, 8, 1), range(0, 24, 1)))):
    unique_coords = combined_df.query(f'resultHour == {hour} and resultWeekday == {day}')
    fig, ax = plt.subplots()
    plt.ion()
    cur = hamburg_df.plot(ax=ax, color ='grey')
    cur = unique_coords.plot(
                       ax=ax,
                       column='average_res',
                       cmap='Blues', 
                       legend=True,
                       norm=plt.Normalize(vmin=1, vmax=50),
                        legend_kwds={"label": "Bikes"}

                      )


    ax.set_xlim(9.7, 10.37)
    ax.set_ylim(53.38, 53.75)
    ax.grid(False)
    ax.set_facecolor("lavender")
    w_day = calendar.day_name[day-1]
    if hour < 10:
        hour_string = "0" + str(hour)
    else:
        hour_string = str(hour)
    ax.set_title(f"Hamburg bike stations at {w_day} {hour_string}:00")
    cur.tick_params(left = False, right = False , labelleft = False ,
            labelbottom = False, bottom = False)
    file_name = f"images/zip_{day}_{hour}.jpg"
    fig.savefig(file_name, dpi=300, bbox_inches='tight')
    plt.close(fig)



100%|██████████████████████████████████████████████████████████████████| 168/168 [01:03<00:00,  2.65it/s]


# Calculating Closest Station

In [13]:
from shapely.ops import nearest_points
list_of_stations = station_df.geometry.unary_union
def get_closest_station(centroid, list_of_stations):
    
     # find the nearest point and return the corresponding Place value
     nearest = station_df.geometry == nearest_points(centroid, list_of_stations)[1]
     return station_df[nearest].copy_geometry.iloc[0]
    
hamburg_df['Nearest'] = hamburg_df.apply(lambda row: get_closest_station(row.geometry.centroid, list_of_stations),
                                         axis=1)
    

NameError: name 'hamburg_df' is not defined

In [20]:
hamburg_df[["plz", "Nearest"]].head()

KeyError: "['Nearest'] not in index"

In [102]:
import folium
m = folium.Map(location=[53.6, 9.98], zoom_start=10)
m

In [ ]:
import folium
m = folium.Map(location=[53.61, 10.01], zoom_start=14)
for _, r in hamburg_df.loc[hamburg_df["plz"] == "22297"].iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    #folium.Popup(r['BoroName']).add_to(geo_j)
    geo_j.add_to(m)
    
for _, r in joined_df.loc[joined_df["plz"] == "22297"].iterrows():
    lat = r['copy_geometry'].y
    lon = r['copy_geometry'].x
    folium.Marker(location=[lat, lon]).add_to(m)
for _, r in hamburg_df.loc[hamburg_df["plz"] == "22297"].iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    
    lat = r['Nearest'].y
    lon = r['Nearest'].x
    folium.Marker(location=[lat, lon], icon=folium.Icon(color='lightgray', icon='Home', prefix='fa')).add_to(m)
m


In [207]:
import geopy.distance
hamburg_df["centroid"] = hamburg_df.geometry.centroid

hamburg_df["centroid"] = hamburg_df["centroid"].to_crs("EPSG:4490")
hamburg_df["Nearest"] = hamburg_df["Nearest"].set_crs("EPSG:4490")
#hamburg_df["avg_dist"] = hamburg_df["centroid"].distance(hamburg_df["Nearest"])

hamburg_df["avg_dist"] = hamburg_df.apply(lambda row: \
                    geopy.distance.geodesic((row["centroid"].y, row["centroid"].x), 
                    (row["Nearest"].y, row["Nearest"].x)).km, 
                                                                              axis=1)

/tmp/ipykernel_107244/4021066020.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hamburg_df["centroid"] = hamburg_df.geometry.centroid


# Graph Calculation

In [16]:
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import networkx as nx
import folium
import osmnx as ox
import matplotlib.pyplot as plt
from descartes import PolygonPatch
from IPython.display import IFrame
from folium.plugins import MarkerCluster
ox.config(log_console=True, use_cache=True)
import json

/Users/felixude/opt/anaconda3/envs/traffic/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [17]:
class DictSmallest(dict):
    def __setitem__(self, key, value):
        if (key not in self) or (key in self and self[key] > value):
            dict.__setitem__(self, key, value)
    def update(self, dict):
        for key, value in dict.items():
            if (key not in self) or (key in self and self[key] > value):
                self[key] =  value
        

In [18]:
unique_station = station_df.groupby('thingID')[["coordinatesY", "coordinatesX"]].min()
unique_station = unique_station.reset_index(drop=True)

all_sub_station = pd.read_csv('input_data/cleaned_stations.csv')

In [19]:
all_sub_station = all_sub_station.rename({"lat": "coordinatesY", "lon": "coordinatesX"}, axis=1)


In [20]:
trip_times = range(1, 51, 1)

iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
iso_colors.reverse()

colormap = cm.LinearColormap(colors=iso_colors)
colormap = colormap.to_step(index=range(0, 51, 5))
    

In [21]:
colormap

In [27]:
%%time

trip_times = range(1, 51, 1)

iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
iso_colors.reverse()

def create_coords_dict(poi_df, filename, center_location ="Hamburg, Germany",  dist=5000):
    G =  ox.graph_from_address(center_location, dist=dist, network_type="walk", simplify=True)
    # 2 - Create nodes geodataframe from Graph network (G)
    gdf_nodes = ox.graph_to_gdfs(G, edges=False)

    list_of_poi = []

    for index, row in poi_df.iterrows():
        list_of_poi.append(ox.distance.nearest_nodes(G, Y=row.coordinatesY, X=row.coordinatesX))

    G = ox.project_graph(G)

    node_distances = DictSmallest()

    for poi in tqdm(list_of_poi):
        tmp_res = nx.shortest_path_length(G, source=poi)
        node_distances.update(tmp_res)
        
    
    def color_mapping_function(val):

        for time, color in zip(trip_times, iso_colors):
            if val < time :
                return color

        return iso_colors[-1]

    coords = {key: {"x": G.nodes[key]["lon"], "y": G.nodes[key]["lat"], "color": color_mapping_function(node_distances[key])} for key in list(G.nodes())}
    with open(f'coords/{filename}.json', 'w') as outfile:
        json.dump(coords, outfile, indent=4)
  

    

def create_folium_plot(coord_file, file_name, caption):

    with open("coords/" + coord_file, "r") as f:
        coords = json.loads(f.read())


    colormap = cm.LinearColormap(colors=iso_colors)
    colormap = colormap.to_step(index=range(0, 51, 5))

    colormap.caption = caption
   
    import folium
    m = folium.Map(location=[53.555, 9.9914], zoom_start=12,prefer_canvas=True, 
                   )



    for val in coords.values():
        folium.Circle(
          location=[val["y"],val["x"]],
            radius=50,
          #popup="Test",
        stroke=False,
        fill=True,
        color = val["color"],
        fill_opacity=0.3,
        interactive=True

       ).add_to(m)
    colormap.add_to(m)
    m.save(f'pages/maps/{file_name}.html')

CPU times: user 4.74 ms, sys: 404 µs, total: 5.15 ms
Wall time: 4.75 ms


In [ ]:
create_coords_dict(unique_station, "station_10000", dist=10000)

In [ ]:
create_coords_dict(all_sub_station, "all_sub_10000", dist=10000)

In [28]:
%%time
caption_bike = "Walking distance in minutes to nearest bike station"
create_folium_plot("bike_10000.json", "hamburg_bike_darker_10000",  caption=caption_bike)

CPU times: user 28.2 s, sys: 672 ms, total: 28.9 s
Wall time: 28.8 s


In [29]:
%%time
caption_bike = "Walking distance in minutes to nearest subway/-urban station"
create_folium_plot("all_sub_10000.json","hamburg_hvv_darker_10000", caption=caption_bike)

CPU times: user 28.3 s, sys: 493 ms, total: 28.8 s
Wall time: 28.7 s


In [ ]:
all_poi = pd.concat([all_sub_station[["coordinatesY", "coordinatesX"]], unique_station])

In [ ]:
create_coords_dict(all_poi, "all_poi_10000", dist=10000)

In [31]:
%%time
caption_bike = "Walking distance in minutes to nearest subway/-urban or bike station"
create_folium_plot("all_poi_10000.json", "hamburg_all_darker_10000", caption=caption_bike)

CPU times: user 28.4 s, sys: 638 ms, total: 29.1 s
Wall time: 28.9 s
